In [4]:
import os
import sys
sys.path.append('../')
from facerec.dataset_generator import DataGenerator
from facerec.siamese_models import get_siamese_model, train_model

In [5]:
IMGS_DIR = r'../data/faces/'
PAIRS_DIR = r'../data/pairs/'
TRAIN_POSITIVE_PAIRS_DIR = os.path.join(PAIRS_DIR, 'train_positive_pairs.csv')
TRAIN_NEGATIVE_PAIRS_DIR = os.path.join(PAIRS_DIR, 'train_negative_pairs.csv')

SEED = 80808
BATCH_SIZE = 32
INPUT_SHAPE = (150, 150, 3)

train_data = DataGenerator(
    positive_pairs_path=TRAIN_POSITIVE_PAIRS_DIR,
    negative_pairs_path=TRAIN_NEGATIVE_PAIRS_DIR,
    images_path=IMGS_DIR,
    input_shape=INPUT_SHAPE,
    batch_size=BATCH_SIZE,
    seed=SEED,
    shuffle=True,
    debug=True
)

VAL_POSITIVE_PAIRS_DIR = os.path.join(PAIRS_DIR, 'val_positive_pairs.csv')
VAL_NEGATIVE_PAIRS_DIR = os.path.join(PAIRS_DIR, 'val_negative_pairs.csv')

val_data = DataGenerator(
    positive_pairs_path=VAL_POSITIVE_PAIRS_DIR,
    negative_pairs_path=VAL_NEGATIVE_PAIRS_DIR,
    images_path=IMGS_DIR,
    input_shape=INPUT_SHAPE,
    batch_size=BATCH_SIZE,
    seed=SEED,
    shuffle=True,
    debug=True
)

TEST_POSITIVE_PAIRS_DIR = os.path.join(PAIRS_DIR, 'test_positive_pairs.csv')
TEST_NEGATIVE_PAIRS_DIR = os.path.join(PAIRS_DIR, 'test_negative_pairs.csv')

test_data = DataGenerator(
    positive_pairs_path=TEST_POSITIVE_PAIRS_DIR,
    negative_pairs_path=TEST_NEGATIVE_PAIRS_DIR,
    images_path=IMGS_DIR,
    input_shape=INPUT_SHAPE,
    batch_size=BATCH_SIZE,
    seed=SEED,
    shuffle=True,
    debug=True
)

print(f'Train data: {BATCH_SIZE * len(train_data)} samples in {len(train_data)} batches')
print(f'Validation data: {BATCH_SIZE * len(val_data)} samples in {len(val_data)} batches')
print(f'Test data: {BATCH_SIZE * len(test_data)} samples in {len(test_data)} batches')

Train data: 24128 samples in 754 batches
Validation data: 3040 samples in 95 batches
Test data: 3040 samples in 95 batches


In [6]:
siamese_model = get_siamese_model('VGG16', input_shape=INPUT_SHAPE)

history = train_model(
    siamese_network=siamese_model,
    model_name='VGG16-150',
    training_dataset_generator=train_data,
    validation_dataset_generator=val_data)

Model: "Pretrained_VGG16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_1 (Dense)             (None, 128)               1048704   
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 128)               0         
                                                                 
Total params: 15,763,392
Trainable params: 1,048,704
Non-trainable params: 14,714,688
_________________________________________________________________
Epoch 1/50
754/754 [==============================] - 335s 430ms/step - loss: 0.2575 - accuracy: 0.5433 - val_loss: 0.2489 - val_accuracy: 0.5613
Epoch 2/50
249/7

KeyboardInterrupt: 

In [ ]:
siamese_model.evaluate(test_data)

In [ ]:
import pandas as pd

hist_df = pd.DataFrame(history.history)

hist_csv_file = r'logs/history_vgg_1809_1.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(5, 5))
plt.axis('equal')
plt.plot(range(1, 51), history.history['loss'], label='Zbiór treningowy')
plt.plot(range(1, 51), history.history['val_loss'], label='Zbiór walidacyjny')
plt.grid()
plt.legend(['train', 'validation'], loc='upper right')
# plt.title('Model loss')
plt.ylabel('Strata kontrastowa')
plt.xlabel('Epoka')
plt.show()

plt.figure(figsize=(5, 5))
plt.axis('equal')
plt.plot(range(1, 51), history.history['accuracy'], label='Zbiór treningowy')
plt.plot(range(1, 51), history.history['val_accuracy'], label='Zbiór walidacyjny')
plt.grid()
plt.legend(['train', 'validation'], loc='upper left')
# plt.title('Model accuracy')
plt.ylabel('Dokładność [%]')
plt.xlabel('Epoka')
plt.show()